In [1]:
import os
import random
from sklearn.utils import compute_class_weight
import numpy as np

dataset_root = "./dataset"
generate_on_the_fly=True

training_file,val_file = os.path.join(dataset_root,"training_list.txt"), os.path.join(dataset_root,"validation_list.txt")

with open(training_file) as f:
    lines = f.readlines()

cat_count = []    
for l in lines:
    path,c = l.split()
    cat_count.append(int(c))
    
category_weights = compute_class_weight('balanced', range(1,6), np.array(cat_count))
print category_weights

[ 1.  1.  1.  1.  1.]


In [2]:
import tflearn, tensorflow as tf
from tflearn.layers.core import input_data, dropout, fully_connected,reshape
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import batch_normalization#local_response_normalization
from tflearn.layers.estimator import regression
from training_utils import ThermalImageAugmentation, thermal_image_preloader

def model(input_placeholder=None):
    tf_data = input_placeholder or tf.placeholder(tf.float32, shape=(None, 224, 224))
    network = input_data(placeholder=tf_data)
    
    network = reshape(network, [-1,224,224,1])
    
    network = batch_normalization(network)
    
    network = conv_2d(network, 96, 7, strides=2, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = batch_normalization(network)
    
    #network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, strides=2, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = batch_normalization(network)
    
    #network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = batch_normalization(network)
    
    network = conv_2d(network, 384, 3, activation='relu')
    network = batch_normalization(network)
    
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = batch_normalization(network)
    
    #network = local_response_normalization(network)
    network = fully_connected(network, 5, activation='softmax')
    
    return network, tf_data

name = "checkpoint-6747"
model_dir = "./checkpoints/2016-12-04_201600/"

graph = tf.Graph()
with graph.as_default():
    # Model variables
    #The network to be used
    with tf.device('/gpu:1'):
        net, X_ph = model()

        init = tf.initialize_all_variables()
        vars_to_restore = []
        for v in tf.all_variables():
            if "is_training" in v.name:
                print v.name
                continue
            vars_to_restore.append(v)
        saver = tf.train.Saver(vars_to_restore)

if generate_on_the_fly:
    X, Y = thermal_image_preloader(training_file, (224,224), 
                             mode='file', categorical_labels=True,
                             normalize=False, grayscale=True,root_path=dataset_root)
    v_X,v_Y = thermal_image_preloader(val_file, (224,224), 
                             mode='file', categorical_labels=True,
                             normalize=False, grayscale=True,root_path=dataset_root)
else:
    import h5py
    h5f = h5py.File(training_h5py_path)
    X = h5f['X']
    Y = h5f['Y']

    v_h5f = h5py.File(val_h5py_path)
    v_X = v_h5f['X']
    v_Y = v_h5f['Y']


# Launch the graph
with tf.Session(graph=graph, config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True)) as sess:
    tflearn.config.is_training(False,sess)
    sess.run(init)
    saver.restore(sess,os.path.join(model_dir,name))
    
    prediction = []
    ground_truth = []
    i=0
    step = 200
    while len(prediction) <len(v_X):
        res = net.eval({X_ph:v_X[i:i+step]})
        for r in res:
            sorted_indexes = sorted(range(5), key=lambda x:r[x], reverse=True)
            prediction.append(sorted_indexes[0])
        i+=step
    
    for r in v_Y:
        sorted_indexes = sorted(range(5), key=lambda x:r[x], reverse=True)
        ground_truth.append(sorted_indexes[0])

        
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ground_truth, prediction)

import numpy as np
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print cm
print cm_norm

BatchNormalization/is_training:0
[[962 136 213  81 162]
 [ 18 465   6  21   0]
 [121 527 582 184  65]
 [ 16 113  23 317   8]
 [ 65  39  51  15 631]]
[[ 0.61904762  0.08751609  0.13706564  0.05212355  0.1042471 ]
 [ 0.03529412  0.91176471  0.01176471  0.04117647  0.        ]
 [ 0.08181204  0.35632184  0.39350913  0.12440838  0.04394861]
 [ 0.03354298  0.23689727  0.04821803  0.66457023  0.01677149]
 [ 0.08114856  0.04868914  0.06367041  0.01872659  0.78776529]]


/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [3]:
import collections, copy

g_err = 0
for jj in range(100):
    a = collections.Counter(ground_truth)
    gt = copy.copy(ground_truth)
    pred = copy.copy(prediction)
    min_class = min(a.values())
    
    while min_class != max(a.values()):
        i = random.choice(range(len(gt)))
        if a[gt[i]] > min_class:
            a[gt[i]] -= 1
            gt.pop(i)
            pred.pop(i)
    err = 0
    for x in range(len(gt)):
        err += 1 if gt[x] != pred[x] else 0
    #print float(err)/len(gt)
    g_err += float(err)/len(gt)
print g_err/100

0.324905660377


In [19]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

#plotly.tools.set_credentials_file()

def truncate(f, n):
    '''Truncates/pads a float f to n decimal places without rounding'''
    s = '{}'.format(f)
    if 'e' in s or 'E' in s:
        return '{0:.{1}f}'.format(f, n)
    i, p, d = s.partition('.')
    return '.'.join([i, (d+'0'*n)[:n]])

def plotly_confusion_matrix(cm, title, norm=False):
    classes=["sleeping", "sitting","standing", "people", "background"]
    annotations = []
    for n, row in enumerate(cm):
        for m, val in enumerate(row):
            text = str("%.2f"%val) if norm else str(val)
            var = cm[n][m]
            annotations.append(
                dict(
                    text=text,
                    x=classes[m], y=classes[n],
                    xref='x1', yref='y1',
                    font=dict(color='white'),
                    showarrow=False)
                )
    data = [
        go.Heatmap(
            x=classes,
            y=classes,
            z=cm,
#            colorscale='Viridis'
        )
    ]
    
    layout = go.Layout(
            title='Confusion Matrix',
            annotations=annotations,
            xaxis=dict(title='Predicted value',),
            yaxis=dict(title='Real value',)
    )
    
    fig = go.Figure(data=data, layout=layout)
    return py.iplot(fig, filename=title)
    
plotly_confusion_matrix(cm, title="confusion_matrix (tf)")

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.



In [21]:
plotly_confusion_matrix(cm_norm, title="confusion_matrix_normalized (tf)", norm=True)

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.

